<div style="width:100%; background-color: #000041"><a target="_blank" href="bit.ly/45U3TvK"><img src="assets/YugabyteDB_DSS-Virtual_LinkedIn-Cover_1584x396.jpg" /></a></div>

<br>

> **Migrate from MySQL to Distributed, Highly Available PostgreSQL!**
>
> Registration is free  [Distributed SQL Summit](bit.ly/45U3TvK).
>

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Review the source database, *which you must do next*
- ☑️ Migration workflow (1)

### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### Import the notebook variables 

> ‼️ Important
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -rMY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

---
# Migration workflow (2)


<div style="width:100%; background-color: #000041"><img src="assets/Migration_Workflow_02.png" /></div>

---
# Export the source schema

In MySQL, there is no difference between a schema and a database. In fact, the `CREATE SCHEMA` command is a synonym for the `CREATE DATABASE` command. To learn more, see [MySQL CREATE DATABASE Statement](https://dev.mysql.com/doc/refman/8.0/en/create-database.html).

In PostgreSQL, a database contains one or more named schemas, which in turn contain tables and other kinds of named objects, including data types, functions, and so on. In fact, the same object name can be used in different schemas without conflict. For example, both `my_schema` and `your_schema` can contain the same table, `table_shared`. 

Unlike databases, schemas are not rigidly separated in PostgreSQL. A user can access objects in any of the schemas in the database they are connected to, as long as they user has the required privileges to do so.

In this step, you will convert the MySQL database to a PostgreSQL format using the `yb-voyager export schema` command. 

Begin by creating an export directory, `export-dir`.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"

NOTEBOOK_VOYAGER_TMP=${1}
echo $NOTEBOOK_VOYAGER_TMP
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir
mkdir $NOTEBOOK_VOYAGER_TMP/export-dir

Next, export the schema using the `yb-voyager` CLI.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01"  "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager export schema --start-clean -y --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
        --source-db-type "MYSQL" \
        --source-db-host ${DB_HOST} \
        --source-db-port  ${DB_PORT} \
        --source-db-name  ${DB_NAME} \
        --source-db-user "ybvoyager" \
        --source-db-password "Yugabyte#1" \ 

List folders and files:
- `reports` contains the generated Schema Analysis Report.
- `schema` contains the source database schema translated to PostgreSQL. The schema is partitioned into smaller files by the schema object type such as tables, views, and so on.
- `data` contains `.TSV` (Tab Separated Values) files that are passed to the `COPY` command on the target database.
- `metainfo` and `temp` are internal to by yb-voyager
- `log/yb-voyager.log` contains log messages

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

ls $NOTEBOOK_VOYAGER_TMP/export-dir

Review the Voyager log.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat $NOTEBOOK_VOYAGER_TMP/export-dir/logs/yb-voyager.log

gp open $NOTEBOOK_VOYAGER_TMP/export-dir/logs/yb-voyager.log

 > Important!
 >
 > 👆 In the above output cell, select **Adjust cell output settings...**. 
 >
 > In the **Settings** tab, in `Notebook > Output: Scrolling`, select the checkbox ✅.
 >
 > In `Notebook > Output: Text Line Limit`, enter `200`.
 >
 > To save, close the **Settings** tab.
 > 
 > Well done! 🙌 

---
# Analyze the exported schema
Generate a Schema Analysis Report using the `yb-voyager analyze-schema` command. 

The report suggests changes to the PostgreSQL schema ao that it is appropriate for YugabyteDB.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
 
# Replace the argument values with those applicable for your migration.
# html, txt, json, or xml
yb-voyager analyze-schema --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir  --output-format txt

ls  $NOTEBOOK_VOYAGER_TMP/export-dir

View the entire analysis report. 

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat  $NOTEBOOK_VOYAGER_TMP/export-dir/reports/report.txt

gp open $NOTEBOOK_VOYAGER_TMP/export-dir/reports/report.txt

The report details the following initial error:
```
Issues:

Error in Object INDEX:
-Object Name: address_location
-Reason: Schema contains gist index which is not supported.
-SQL Statement: CREATE INDEX address_location ON address USING gist(location);
-File Path: /home/gitpod/export-dir/schema/tables/INDEXES_table.sql
-Github Issue Link: https://github.com/YugaByte/yugabyte-db/issues/1337
```

View the `INDEXES_table.sql` file.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/INDEXES_table.sql

gp open $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/INDEXES_table.sql

---
# Modify the exported schema
Using the report recommendations, manually change the exported schema. 

In this case, since YugabyteDB does not currently support `GIST` indexes, it makes sense to not create this index.  So the procedure here is to simply delete the `CREATE INDEX address_location` statement.

You can do this in the open editor for `INDEXES_table.sql` or simply run the following `sed` command.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

sed -i '/gist/d'  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/INDEXES_table.sql

Verify the deletion of the line.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/INDEXES_table.sql

---
# Rinse & Repeat: Analyze the modified schema

After making the manual changes, re-run the `yb-voyager analyze-schema` command and generate a new report. Rinse and repeat until the generated report contains no known issues.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
 
# Replace the argument values with those applicable for your migration.
# html, txt, json, or xml
yb-voyager analyze-schema --export-dir  $NOTEBOOK_VOYAGER_TMP/export-dir  --output-format txt

cat  $NOTEBOOK_VOYAGER_TMP/export-dir/reports/report.txt

gp open  $NOTEBOOK_VOYAGER_TMP/export-dir/reports/report.txt

No errors! Nicely done 🙌 

But before moving on, it is important to a look at the primary key definitions. YugabyteDB is a distributed SQL database. YugabyteDB automatically shards data in a table into tablets. The primary key in YSQL defines the shard key, also known as a partition key. In fact a primary key can consist of a partition key and clustering key. A partition key can implement one of two sharding strategies: consistent hash sharding or range sharding. By default, this is consistent hash sharding. 

So, as a next step, review the `table.sql` file. In particular, review the table definitions for the primary key. 

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
 
cat   $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

gp open  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql


One observation is to make is that the Primary Key columns in the schema export are of the type `serial`. This means that the source MYSQL database used auto-increment primary key of the type `int` or similar. YugabyteDB supports this using the `SERIAL` datatype.

In YugabyteDB, a `SERIAL` type is a sequence. Sequence are cached on the YB-TServer. There are numerous strategies to help with sequence caching that can be managed post migration.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
 
cat  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql | grep 'PRIMARY KEY'
cat  $NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql | grep -B 1 'serial'

### Sequence migration considerations

The auto-increment column in MySQL migrates to YugabyteDB as a normal column of the type `serial` backed by a sequence. 

Sequence migration consists of two steps: sequence creation and setting the resume value. The resume value refers to the `NEXTVAL` of a sequence.

YugabyteDB Voyager creates the DDL for the sequence objects as a result of the `export schema` command. These resume values are then set on the target database just after the data is imported for all tables.

Unlike PostgreSQL, YugabyteDB is a distributed database and clients can connect to any node in a universe. For this reason, sequences in YugabyteDB often needs additional tuning.

---

## 😊 Next up!
Continue your learning by opening the next notebook, `04_Migration_Workflow_Schema_2.ipynb`. 

Or, if using GitPod, run the following cell:

In [ ]:
%%bash
gp open '04_Migration_Workflow_Schema_2.ipynb'